# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative-algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

W0822 07:08:24.412000 330893 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 07:08:24.412000 330893 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 07:08:25] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=34404, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=11812

[2025-08-22 07:08:25] Using default HuggingFace chat template with detected content format: string


W0822 07:08:33.059000 331107 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 07:08:33.059000 331107 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0822 07:08:33.065000 331108 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 07:08:33.065000 331108 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 07:08:33] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-22 07:08:33] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=53.65 GB): 100%|██████████| 4/4 [00:04<00:00,  1.23s/it]
[2025-08-22 07:08:47] Capture draft cuda graph end. Time elapsed: 5.94 s. mem usage=0.14 GB. avail mem=53.64 GB.
[2025-08-22 07:08:47] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=53.64 GB


Capturing batches (bs=1 avail_mem=53.56 GB):   0%|          | 0/4 [00:00<?, ?it/s]

Capturing batches (bs=1 avail_mem=53.56 GB): 100%|██████████| 4/4 [00:00<00:00, 19.35it/s]
[2025-08-22 07:08:48] Capture draft extend cuda graph end. Time elapsed: 0.75 s. mem usage=0.08 GB. avail mem=53.56 GB.
[2025-08-22 07:08:48] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=53.56 GB
[2025-08-22 07:08:48] INFO:     Started server process [330893]
[2025-08-22 07:08:48] INFO:     Waiting for application startup.
[2025-08-22 07:08:48] INFO:     Application startup complete.
[2025-08-22 07:08:48] INFO:     Uvicorn running on http://127.0.0.1:34404 (Press CTRL+C to quit)


[2025-08-22 07:08:48] INFO:     127.0.0.1:59430 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-22 07:08:49] INFO:     127.0.0.1:59434 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-22 07:08:49] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-22 07:08:49] INFO:     127.0.0.1:59450 - "POST /generate HTTP/1.1" 200 OK
[2025-08-22 07:08:49] The server is fired up and ready to roll!


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-22 07:08:53] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-22 07:08:53] INFO:     127.0.0.1:50768 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

W0822 07:09:00.846000 331913 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 07:09:00.846000 331913 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 07:09:01] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=33706, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=3073269

[2025-08-22 07:09:01] Using default HuggingFace chat template with detected content format: string


W0822 07:09:09.677000 332130 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 07:09:09.677000 332130 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0822 07:09:09.751000 332129 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 07:09:09.751000 332129 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 07:09:10] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-22 07:09:10] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=53.49 GB): 100%|██████████| 4/4 [00:06<00:00,  1.66s/it]
[2025-08-22 07:09:35] Capture draft cuda graph end. Time elapsed: 7.25 s. mem usage=0.23 GB. avail mem=53.45 GB.
[2025-08-22 07:09:35] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=53.45 GB


Capturing batches (bs=1 avail_mem=53.35 GB): 100%|██████████| 4/4 [00:00<00:00, 83.66it/s]
[2025-08-22 07:09:36] Capture draft extend cuda graph end. Time elapsed: 0.67 s. mem usage=0.10 GB. avail mem=53.35 GB.
[2025-08-22 07:09:36] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=53.35 GB
[2025-08-22 07:09:36] INFO:     Started server process [331913]
[2025-08-22 07:09:36] INFO:     Waiting for application startup.


[2025-08-22 07:09:36] INFO:     Application startup complete.
[2025-08-22 07:09:36] INFO:     Uvicorn running on http://127.0.0.1:33706 (Press CTRL+C to quit)


[2025-08-22 07:09:37] INFO:     127.0.0.1:50150 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-22 07:09:37] INFO:     127.0.0.1:50162 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-22 07:09:37] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 07:09:37] INFO:     127.0.0.1:50176 - "POST /generate HTTP/1.1" 200 OK
[2025-08-22 07:09:37] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-22 07:09:42] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-22 07:09:42] INFO:     127.0.0.1:50180 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

W0822 07:09:48.940000 333206 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 07:09:48.940000 333206 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 07:09:49] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=39033, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, rand

[2025-08-22 07:09:50] Using default HuggingFace chat template with detected content format: string


W0822 07:09:57.413000 333421 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 07:09:57.413000 333421 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0822 07:09:57.483000 333422 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 07:09:57.483000 333422 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 07:09:57] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=2 avail_mem=58.27 GB):   0%|          | 0/4 [00:00<?, ?it/s]

Capturing batches (bs=1 avail_mem=58.25 GB): 100%|██████████| 4/4 [00:00<00:00, 13.60it/s]
[2025-08-22 07:10:26] Capture draft extend cuda graph end. Time elapsed: 1.03 s. mem usage=0.10 GB. avail mem=58.25 GB.
[2025-08-22 07:10:26] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=8192, available_gpu_mem=58.25 GB


[2025-08-22 07:10:27] INFO:     Started server process [333206]
[2025-08-22 07:10:27] INFO:     Waiting for application startup.
[2025-08-22 07:10:27] INFO:     Application startup complete.
[2025-08-22 07:10:27] INFO:     Uvicorn running on http://127.0.0.1:39033 (Press CTRL+C to quit)


[2025-08-22 07:10:27] INFO:     127.0.0.1:56064 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-22 07:10:28] INFO:     127.0.0.1:56076 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-22 07:10:28] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-22 07:10:28] INFO:     127.0.0.1:56080 - "POST /generate HTTP/1.1" 200 OK
[2025-08-22 07:10:28] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-22 07:10:32] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-22 07:10:32] INFO:     127.0.0.1:56086 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative-algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

W0822 07:10:39.683000 334268 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 07:10:39.683000 334268 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 07:10:40] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=38077, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_see

[2025-08-22 07:10:41] Using default HuggingFace chat template with detected content format: string


W0822 07:10:48.077000 334484 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 07:10:48.077000 334484 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0822 07:10:48.190000 334483 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 07:10:48.190000 334483 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 07:10:48] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=58.24 GB): 100%|██████████| 4/4 [00:04<00:00,  1.18s/it]
[2025-08-22 07:11:12] Capture draft cuda graph end. Time elapsed: 5.13 s. mem usage=0.23 GB. avail mem=58.20 GB.
[2025-08-22 07:11:12] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=58.20 GB


Capturing batches (bs=1 avail_mem=58.09 GB): 100%|██████████| 4/4 [00:00<00:00, 93.80it/s]
[2025-08-22 07:11:12] Capture draft extend cuda graph end. Time elapsed: 0.53 s. mem usage=0.11 GB. avail mem=58.09 GB.
[2025-08-22 07:11:12] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=58.09 GB


[2025-08-22 07:11:13] INFO:     Started server process [334268]
[2025-08-22 07:11:13] INFO:     Waiting for application startup.
[2025-08-22 07:11:13] INFO:     Application startup complete.
[2025-08-22 07:11:13] INFO:     Uvicorn running on http://127.0.0.1:38077 (Press CTRL+C to quit)


[2025-08-22 07:11:13] INFO:     127.0.0.1:52660 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-22 07:11:14] INFO:     127.0.0.1:52670 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-22 07:11:14] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-22 07:11:14] INFO:     127.0.0.1:52684 - "POST /generate HTTP/1.1" 200 OK
[2025-08-22 07:11:14] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-22 07:11:18] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-22 07:11:19] INFO:     127.0.0.1:52700 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

W0822 07:11:26.268000 335288 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 07:11:26.268000 335288 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 07:11:27] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=30851, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=250602686, constrained_jso

[2025-08-22 07:11:27] Using default HuggingFace chat template with detected content format: string


W0822 07:11:34.364000 335504 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 07:11:34.364000 335504 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0822 07:11:34.666000 335503 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 07:11:34.666000 335503 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


...

Capturing batches (bs=1 avail_mem=59.45 GB): 100%|██████████| 4/4 [00:01<00:00,  2.08it/s]
[2025-08-22 07:11:46] Capture draft cuda graph end. Time elapsed: 2.54 s. mem usage=0.01 GB. avail mem=59.45 GB.
[2025-08-22 07:11:46] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=59.45 GB


Capturing batches (bs=1 avail_mem=59.35 GB): 100%|██████████| 4/4 [00:00<00:00, 42.92it/s]
[2025-08-22 07:11:47] Capture draft extend cuda graph end. Time elapsed: 0.78 s. mem usage=0.09 GB. avail mem=59.35 GB.
[2025-08-22 07:11:47] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=32768, available_gpu_mem=59.35 GB


[2025-08-22 07:11:48] INFO:     Started server process [335288]
[2025-08-22 07:11:48] INFO:     Waiting for application startup.
[2025-08-22 07:11:48] INFO:     Application startup complete.
[2025-08-22 07:11:48] INFO:     Uvicorn running on http://0.0.0.0:30851 (Press CTRL+C to quit)


[2025-08-22 07:11:49] INFO:     127.0.0.1:42430 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-22 07:11:49] INFO:     127.0.0.1:42432 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-22 07:11:49] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 07:11:49] INFO:     127.0.0.1:42440 - "POST /generate HTTP/1.1" 200 OK
[2025-08-22 07:11:49] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-22 07:11:54] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 07:11:54] Decode batch. #running-req: 1, #token: 91, token usage: 0.00, accept len: 1.76, cuda graph: True, gen throughput (token/s): 10.62, #queue-req: 0, 


[2025-08-22 07:11:54] Decode batch. #running-req: 1, #token: 161, token usage: 0.01, accept len: 1.75, cuda graph: True, gen throughput (token/s): 178.63, #queue-req: 0, 


[2025-08-22 07:11:55] Decode batch. #running-req: 1, #token: 233, token usage: 0.01, accept len: 1.80, cuda graph: True, gen throughput (token/s): 183.21, #queue-req: 0, 


[2025-08-22 07:11:55] Decode batch. #running-req: 1, #token: 304, token usage: 0.01, accept len: 1.77, cuda graph: True, gen throughput (token/s): 180.58, #queue-req: 0, 


[2025-08-22 07:11:56] Decode batch. #running-req: 1, #token: 376, token usage: 0.02, accept len: 1.80, cuda graph: True, gen throughput (token/s): 183.04, #queue-req: 0, 


[2025-08-22 07:11:56] Decode batch. #running-req: 1, #token: 450, token usage: 0.02, accept len: 1.85, cuda graph: True, gen throughput (token/s): 188.09, #queue-req: 0, 
[2025-08-22 07:11:56] INFO:     127.0.0.1:39798 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).